In [1]:
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
client = Client(n_workers = 16)
client

Client Scheduler: tcp://127.0.0.1:42263 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 67.53 GB


### Combining all CSV files that match 'Youtube*.csv'

In [3]:
%%time

large_yt_dd = dd.read_csv('Youtube*.csv')

CPU times: user 14.3 ms, sys: 6.85 ms, total: 21.2 ms
Wall time: 30.5 ms


In [4]:
large_yt_dd

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
npartitions=5,,,,,
,object,object,object,object,int64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [5]:
large_yt_dd.shape

(Delayed('int-2fbf0f0d-3efd-4250-a790-e2e434e55679'), 5)

In [6]:
len(large_yt_dd)

1956

### Spam vs Not Spam (CLASS = 1 => Spam)

In [8]:
large_yt_dd['CLASS'].value_counts().compute()

1    1005
0     951
Name: CLASS, dtype: int64

In [14]:
large_yt_dd['CONTENT'].head()

0    Huh, anyway check out this you[tube] channel: ...
1    Hey guys check out my new channel and our firs...
2               just for test I have to say murdev.com
3     me shaking my sexy ass on my channel enjoy ^_^ ﻿
4              watch?v=vtaRGgvGtWQ   Check this out .﻿
Name: CONTENT, dtype: object

In [19]:
large_yt_dd['content'] = '0'
large_yt_dd

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,content
npartitions=5,,,,,,
,object,object,object,object,int64,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [22]:
def make_lower(s):
    return s.lower()

large_yt_dd['content'] = large_yt_dd['CONTENT'].apply(make_lower)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/dask/dataframe/core.py:2285: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [23]:
large_yt_dd['content'].head()

0    huh, anyway check out this you[tube] channel: ...
1    hey guys check out my new channel and our firs...
2               just for test i have to say murdev.com
3     me shaking my sexy ass on my channel enjoy ^_^ ﻿
4              watch?v=vtarggvgtwq   check this out .﻿
Name: content, dtype: object

In [29]:
def has_check(s):
    if "check" in s:
        return 1
    else:
        return 0

large_yt_dd['check'] = 0

large_yt_dd['check'] = large_yt_dd['content'].apply(has_check)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/dask/dataframe/core.py:2285: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [39]:
large_yt_dd['check'].value_counts().compute()

0    1476
1     480
Name: check, dtype: int64

In [42]:
help(large_yt_dd.pivot_table)

Help on method pivot_table in module dask.dataframe.core:

pivot_table(index=None, columns=None, values=None, aggfunc='mean') method of dask.dataframe.core.DataFrame instance
    Create a spreadsheet-style pivot table as a DataFrame. Target ``columns``
    must have category dtype to infer result's ``columns``.
    ``index``, ``columns``, ``values`` and ``aggfunc`` must be all scalar.
    
    Parameters
    ----------
    values : scalar
        column to aggregate
    index : scalar
        column to be index
    columns : scalar
        column to be columns
    aggfunc : {'mean', 'sum', 'count'}, default 'mean'
    
    Returns
    -------
    table : DataFrame



In [45]:
large_yt_dd=large_yt_dd.categorize("CLASS")
large_yt_dd=large_yt_dd.categorize("check")

ddpt = large_yt_dd.pivot_table(
            values = 'COMMENT_ID',
            index = 'CLASS',
            columns = 'check',
            aggfunc = 'count')

ddpt.compute()

check,1,0
CLASS,,
1,461.0,544.0
0,19.0,932.0


#### 461 Spam messages have the word "check", 544 Spam messages don't.

#### 19 Legit messages have the word "check", 932 Legit messages don't. 

## Part 2

    In choosing a partiular platform, library or language, it is important to match it to the needs of the project. In dealing with managable sized data, I would choose tools like Python 3 (Pandas, Numpy) and SQL because they provide good performance in a relatively short period of time. For tasks that require more computing and processing power above and beyond the normal capabilities of Python 3 and SQL, I would take advantage of "Big Data" tools like Spark and MapReduce and use libraries like Numba and Dask (a distributed DataFrame). I would also employ the use of AWS which allows one to work on a computationally long program with more RAM and cores than on my local device. One example of where I would use AWS and these "Big Data" tools is genomics data analysis where the problems require more RAM and more cores than on the vast majority of local machines. Rather than possibly burning out my local computer and hanging in a program for hours/days, I can instead exploit AWS's computing power (RAM, cores) to get the results I seek much faster.